In [ ]:
import MySQLdb
from pandas.io import sql
import pandas as pd
import numpy as np
conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor()

In [ ]:
df_visit = pd.read_sql('select * from `visit`', con = conn)
df_visit

In [ ]:
df_visit[df_visit['patient_id']==10467237]

In [ ]:
df_visit_type = pd.read_sql('select * from `visit_type`', con = conn)

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="Soic@2022Capstone", db="mimic4") 
mimiccursor = mimicconn.cursor()

In [ ]:
admissions_mimic = pd.read_sql('select * from admissions', con = mimicconn) 
admissions_mimic

In [ ]:
admissions_visit = pd.read_sql('select distinct(admission_type) from admissions', con = mimicconn) 
admissions_visit

In [ ]:
import uuid
admissions_visit['date_created'] = '2022-01-01'
admissions_visit['uuid'] = [uuid.uuid4() for x in range(len(admissions_visit))]
admissions_visit['creator'] = 1

In [ ]:
admissions_visit.rename(columns={"admission_type": "name"})

In [ ]:
dict_ = pd.Series(df_visit_type.visit_type_id.values,index=df_visit_type.name).to_dict()
dict_

# insert columns from admissions to visit table

In [ ]:
admissions_mimic_copy = admissions_mimic[['subject_id', 'hadm_id', 'admittime', 'dischtime', 'admission_type']]
admissions_mimic_copy['visit_type_id'] = admissions_mimic_copy['admission_type'].map(dict_)

In [ ]:
admissions_mimic_copy['date_created'] = '2022-01-01'
admissions_mimic_copy['uuid'] = [uuid.uuid4() for x in range(len(admissions_mimic_copy))]
admissions_mimic_copy['creator'] = 1
admissions_mimic_copy

In [ ]:
admissions_mimic_copy=admissions_mimic_copy.rename(columns={"subject_id": "patient_id", "hadm_id": "visit_id", "admittime": "date_started", "dischtime": "date_stopped"}) 
admissions_mimic_copy = admissions_mimic_copy.drop(columns=['admission_type']) 
admissions_mimic_copy 

In [ ]:
admissions_mimic_copy['visit_id'].nunique()

In [ ]:
admissions_mimic_copy.dtypes

# getting Indication_concept_id from Concept table

In [ ]:
# hadmid_conceptid = pd.read_sql('SELECT c.concept_id, diag.hadm_id FROM openmrs.concept c, mimic4.d_icd_diagnoses d, mimic4.diagnoses_icd diag where c.description = d.long_title and diag.icd_code = d.icd_code and diag.seq_num=1')


In [ ]:
hadmid_conceptid = pd.read_csv('hadm_id and concept_id.csv')
hadmid_conceptid = hadmid_conceptid.rename(columns={"hadm_id": "visit_id"})
hadmid_conceptid

In [ ]:
hadmid_conceptid['visit_id'].nunique()

In [ ]:
new_vist_admissions = pd.merge(admissions_mimic_copy,hadmid_conceptid, on= 'visit_id', how='left')
new_vist_admissions

In [ ]:
new_vist_admissions_1 = new_vist_admissions.drop_duplicates(subset ='visit_id')
new_vist_admissions_1 = new_vist_admissions_1.rename({'concept_id':'indication_concept_id'}, axis = 1)
new_vist_admissions_1

In [ ]:
column_names = ['visit_id', 'patient_id', 'visit_type_id', 'date_started', 'date_stopped','indication_concept_id','creator', 'date_created', 'uuid']
final_visit_df = new_vist_admissions_1.reindex(columns=column_names)
final_visit_df

In [ ]:
final_visit_df = final_visit_df.replace({np.nan: None})
final_visit_df

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in final_visit_df.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in final_visit_df.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `visit` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 

In [ ]:
# UPDATE openmrs.visit ov SET indication_concept_id = (select concept_id from openmrs.concept
# WHERE short_name = (select icd_code from mimic4.diagnoses_icd mdiag
# WHERE mdiag.hadm_id = ov.visit_id AND mdiag.seq_num = 1)) WHERE ;

In [ ]:
# select COUNT(c.concept_id) from mimic4.diagnoses_icd mdiag, openmrs.visit ov, openmrs.concept c
# WHERE mdiag.hadm_id = ov.visit_id AND mdiag.seq_num = 1 AND c.short_name=mdiag.icd_code